### 🔨 Setup

⚠ In order to run this notebook you need two separate services:

- ChromaDB: `chroma run --path ./data/chromadb --port 7000`
- vLLM:     `vllm serve <your-model> --chat-template <your-template>`

In [1]:
from pathlib import Path

LOCAL_OPENAI_API_URI = "http://localhost:8000/v1"
LOCAL_CHROMA_URI = "localhost:7000"
OPENAI_API_KEY = "EMPTY"
DATA_PATH = "./data"
DOCS_PATH = Path(DATA_PATH) / "processed_docs"

# Change to the root of the repo (RUN ONLY ONCE!)
%cd ..
print(Path.cwd())

/home/ubuntu/code/LLM-exploratory
/home/ubuntu/code/LLM-exploratory


/home/ubuntu/code/LLM-exploratory/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### 🔌 OpenAI-like API

In [8]:
from openai import OpenAI

# Check what model is available from the API
client = OpenAI(api_key=OPENAI_API_KEY, base_url=LOCAL_OPENAI_API_URI)
available_llms = client.models.list().data
print(f"Available models:")
print(" - " + " - ".join(llm.id for llm in available_llms))

Available models:
 - neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a8


In [9]:
chat_response = client.chat.completions.create(
    model=available_llms[0].id,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the best thing about LLMs?"},
    ],
)
print("Chat response:", chat_response.choices[0].message.content)

Chat response: Large Language Models (LLMs) have several impressive features, but some of the best things about them include:

1. **Multilingual Support**: LLMs can understand and generate text in multiple languages, making them a powerful tool for global communication and translation.
2. **Contextual Understanding**: LLMs can comprehend the context of a conversation or text, allowing them to provide more accurate and relevant responses.
3. **Creative Writing**: LLMs can generate creative content, such as stories, poems, and dialogues, making them a valuable tool for writers and content creators.
4. **Conversational Dialogue**: LLMs can engage in natural-sounding conversations, making them a great tool for customer service, chatbots, and language learning.
5. **Knowledge Retrieval**: LLMs can quickly retrieve and provide information on a wide range of topics, making them a valuable resource for research and learning.
6. **Sentiment Analysis**: LLMs can analyze text to determine the sen

### ❓ DocQA

In [3]:
import json


from llm.xplore.rag.qa import DocQAClient

llm_model_id = available_llms[0].id

# Init the QA Client
print(f"Initiliazing DocQAClient for OpenAI-like API. LLM='{llm_model_id}'")
qa_client = DocQAClient(
    chroma_uri=LOCAL_CHROMA_URI,
    openai_api_url=LOCAL_OPENAI_API_URI,
    openai_api_key=OPENAI_API_KEY,
)

Initiliazing DocQAClient for OpenAI-like API. LLM='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a8'


/home/ubuntu/code/LLM-exploratory/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/ubuntu/code/LLM-exploratory/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/ubuntu/code/LLM-exploratory/llm/xplore/rag/qa.py:181: LangChainDeprecationWarning: The function `create_structured_output_chain` was deprecated in LangChain 0.1.1 and will be removed in 1.0. Use :meth:`~ChatOpenAI.with_structured_output` instead.
  chain = create_st

#### 📚 Indexing

In [ ]:
from llm.xplore.helpers import collect_files

# Index some documents
doc_files = collect_files(DOCS_PATH)
print(f"📚 Total files: {len(doc_files)}")

for doc_file in doc_files:
    print(f"📖 Indexing doc '{doc_file.name}'")
    doc_data = json.loads(doc_file.open().read())
    qa_client.index_document(
        doc_pages=doc_data["text"],
        doc_meta={
            "id": doc_data["sha1"],
            "name": doc_data["name"],
            "type": doc_data["type"],
        },
    )

#### 🧲 Retrieval

In [4]:
# Check for indexed documents
qa_client.list_documents()

{'132d216ac71cd53e993251450243e588f548bf1d',
 '17f14f0a89df3ba96cc4966588682288e07fbc8d',
 '2a0ad74b29c929b9461f9bd2266518190fed4e27',
 '78944398eb5ef89fba1e4e7cd512066619982766',
 'b2e85f84c2ea58571e5e270bf57f0ea98306ddb0',
 'c90d52f64fb051f9d3687d03d3e6fa5e6bddfb5b',
 'cf0fae563ffc10db0ecb2be19466b38afb2dc8cc',
 'ebfc841fbb0e2126d6cc29df0aa73d0454711e69'}

#### 🦜 Generate answers

In [ ]:
# Query the DB and generate some answers
from llm.xplore.rag.schemas import QuestionType


qa_client.get_answers(
    queries=["Who are the parties to this agreement?"],
    query_types=[QuestionType.EXTRACTIVE],
    filters={"id": "132d216ac71cd53e993251450243e588f548bf1d"},
    print_sources=True
)